###  ABS to Tidydata

In [1]:
from gssutils import *

scraper = Scraper('https://www.ons.gov.uk/businessindustryandtrade/' + \
                  'business/businessservices/datasets/annualbusinesssurveyimportersandexporters')
scraper

## Annual Business Survey importers and exporters

Importers and exporters of goods and services in Great Britain by employment size, turnover size, ownership and age, 2015 to 2017.

### Distributions

1. Annual Business Survey importers and exporters ([MS Excel Spreadsheet](https://www.ons.gov.uk/file?uri=/businessindustryandtrade/business/businessservices/datasets/annualbusinesssurveyimportersandexporters/current/absimportersandexporterssummarynov2018.xlsx))


In [2]:
sheets = scraper.distribution().as_databaker()

In [3]:
import re
tab_name_re = re.compile(r'^([0-9]{4}) (.*)$')
tidy = pd.DataFrame()

for sheet in sheets[1:-1]:
    name_match = tab_name_re.match(sheet.name)
    assert name_match, "sheet name doesn't match regex"
    for breakdown in ['Detailed employment', 'Employment', 'Ownership', 'Turnover', 'Age']:
        year = HDimConst('Year', name_match.group(1))
        trade = HDimConst('Trade', name_match.group(2).strip())
        breakdown_on_down = sheet.filter(starts_with(breakdown)).fill(DOWN).expand(RIGHT).is_not_blank()
        breakdown_obs = breakdown_on_down - \
            breakdown_on_down.filter(contains_string('Total')).expand(DOWN).expand(RIGHT) - \
            sheet.filter(starts_with(breakdown)).fill(DOWN)
        classifiers = sheet.filter(starts_with(breakdown)).fill(DOWN).is_not_blank()
        classifiers = classifiers - classifiers.filter(contains_string('Total')).expand(DOWN)
        classifiers = HDim(classifiers, breakdown, DIRECTLY, LEFT)
        classifiers.AddCellValueOverride('2 to9', '2 to 9')
        import_export = sheet.filter(starts_with(breakdown)).fill(RIGHT).is_not_blank()
        import_export = HDim(import_export, 'Import/Export', DIRECTLY, UP)
        import_export.AddCellValueOverride('Businesses 4', 'Businesses')
        import_export.AddCellValueOverride('Exporter and/or Importer 7', 'Exporter and/or Importer')
        measure = sheet.filter(starts_with(breakdown)).shift(UP).fill(RIGHT).is_not_blank()
        measure = HDim(measure, 'Measure Type', CLOSEST, LEFT)
        measure.AddCellValueOverride('Number of 5', 'Count')
        measure.AddCellValueOverride('% 6', 'Proportion of all Business')
        tidy = tidy.append(ConversionSegment(breakdown_obs, [classifiers, import_export, year, trade, measure]).topandas(), sort=True)
        savepreviewhtml([breakdown_obs, classifiers, import_export, measure])
        #break
    #break

tidy

,Age,Detailed employment,Employment,Import/Export,Measure Type,OBS,Ownership,Trade,Turnover,Year
0,NaN,1,NaN,Businesses,Count,1067100.0,NaN,Goods and Services,NaN,2017
1,NaN,1,NaN,Exporters,Count,89800.0,NaN,Goods and Services,NaN,2017
2,NaN,1,NaN,Importers,Count,72000.0,NaN,Goods and Services,NaN,2017
3,NaN,1,NaN,Exporter and Importer,Count,39600.0,NaN,Goods and Services,NaN,2017
4,NaN,1,NaN,Exporter and/or Importer,Count,122200.0,NaN,Goods and Services,NaN,2017
5,NaN,1,NaN,Exporters,Proportion of all Business,8.4,NaN,Goods and Services,NaN,2017
6,NaN,1,NaN,Importers,Proportion of all Business,6.8,NaN,Goods and Services,NaN,2017
7,NaN,1,NaN,Exporter and Importer,Proportion of all Business,3.7,NaN,Goods and Services,NaN,2017
8,NaN,1,NaN,Exporter and/or Importer,Proportion of all Business,11.5,NaN,Goods and Services,NaN,2017
9,NaN,2 to 9,NaN,Businesses,Count,1059700.0,NaN,Goods and Services,NaN,2017


Check for duplicate rows

In [4]:
assert tidy.duplicated().sum() == 0, 'duplicate rows'

"Employment" is the parent of "Detailed employment".

Also, the class "250 and over" is repeated in each, so we need to drop the duplicates. However, there appear to be some discrepancies.

In [5]:
duplicate_label = '250 and over'
emp_250 = tidy[tidy['Employment'] == duplicate_label].drop(columns=['Employment', 'Detailed employment']).reset_index(drop=True)
detailed_emp_250 = tidy[tidy['Detailed employment'] == duplicate_label].drop(columns=['Employment', 'Detailed employment']).reset_index(drop=True)
assert emp_250.size > 0
assert detailed_emp_250.size > 0
#assert emp_250.equals(detailed_emp_250)
merged = emp_250.merge(detailed_emp_250, indicator=True, how='outer')

display(merged[merged['_merge'] == 'right_only'])

tidy = tidy[tidy['Detailed employment'] != '250 and over'].reset_index(drop=True)

,Age,Import/Export,Measure Type,OBS,Ownership,Trade,Turnover,Year,_merge


We need to merge them and also list their values so that we can create a codelist.

In [6]:
display(tidy['Employment'].unique())
display(tidy['Detailed employment'].unique())
tidy['Employees'] = tidy.apply(lambda x: x['Employment'] if pd.notnull(x['Employment']) else x['Detailed employment'], axis=1)
tidy = tidy.drop(columns=['Employment', 'Detailed employment'])
tidy

array([nan, '1 to 49', '50 to 249', '250 and over'], dtype=object)

array(['1', '2 to 9', '10 to 19', '20 to 49', '50 to 99', '100 to 249',
       nan], dtype=object)

,Age,Import/Export,Measure Type,OBS,Ownership,Trade,Turnover,Year,Employees
0,NaN,Businesses,Count,1067100.0,NaN,Goods and Services,NaN,2017,1
1,NaN,Exporters,Count,89800.0,NaN,Goods and Services,NaN,2017,1
2,NaN,Importers,Count,72000.0,NaN,Goods and Services,NaN,2017,1
3,NaN,Exporter and Importer,Count,39600.0,NaN,Goods and Services,NaN,2017,1
4,NaN,Exporter and/or Importer,Count,122200.0,NaN,Goods and Services,NaN,2017,1
5,NaN,Exporters,Proportion of all Business,8.4,NaN,Goods and Services,NaN,2017,1
6,NaN,Importers,Proportion of all Business,6.8,NaN,Goods and Services,NaN,2017,1
7,NaN,Exporter and Importer,Proportion of all Business,3.7,NaN,Goods and Services,NaN,2017,1
8,NaN,Exporter and/or Importer,Proportion of all Business,11.5,NaN,Goods and Services,NaN,2017,1
9,NaN,Businesses,Count,1059700.0,NaN,Goods and Services,NaN,2017,2 to 9


Fill NaN with top values.

In [7]:
tidy.fillna(value={'Age': 'Any', 'Ownership': 'Any', 'Turnover': 'Any', 'Employees': 'Any', }, inplace=True)
tidy

,Age,Import/Export,Measure Type,OBS,Ownership,Trade,Turnover,Year,Employees
0,Any,Businesses,Count,1067100.0,Any,Goods and Services,Any,2017,1
1,Any,Exporters,Count,89800.0,Any,Goods and Services,Any,2017,1
2,Any,Importers,Count,72000.0,Any,Goods and Services,Any,2017,1
3,Any,Exporter and Importer,Count,39600.0,Any,Goods and Services,Any,2017,1
4,Any,Exporter and/or Importer,Count,122200.0,Any,Goods and Services,Any,2017,1
5,Any,Exporters,Proportion of all Business,8.4,Any,Goods and Services,Any,2017,1
6,Any,Importers,Proportion of all Business,6.8,Any,Goods and Services,Any,2017,1
7,Any,Exporter and Importer,Proportion of all Business,3.7,Any,Goods and Services,Any,2017,1
8,Any,Exporter and/or Importer,Proportion of all Business,11.5,Any,Goods and Services,Any,2017,1
9,Any,Businesses,Count,1059700.0,Any,Goods and Services,Any,2017,2 to 9


Show the range of the codes and check for duplicated rows.

In [8]:
from IPython.core.display import HTML
for col in tidy:
    if col not in ['OBS']:
        display(HTML(f'<h2>{col}</h2>'))
        display(tidy[col].unique())
dups = tidy.duplicated()
display(dups.sum())
tidy[dups]

array(['Any', '<2', '2-<4', '4-<10', '10-<20', '20+'], dtype=object)

array(['Businesses', 'Exporters', 'Importers', 'Exporter and Importer',
       'Exporter and/or Importer'], dtype=object)

array(['Count', 'Proportion of all Business'], dtype=object)

array(['Any', 'UK', 'Foreign'], dtype=object)

array(['Goods and Services', 'Goods', 'Services'], dtype=object)

array(['Any', '<1000', '1000 - 4999', '5000 - 9999', '10,000 - 24,999',
       '25,000 - 49,999', '50,000 - 99,999', '100,000 - 249,999',
       '250,000 - 499,999', '500,000 +'], dtype=object)

array(['2017', '2016', '2015'], dtype=object)

array(['1', '2 to 9', '10 to 19', '20 to 49', '50 to 99', '100 to 249',
       '1 to 49', '50 to 249', '250 and over', 'Any'], dtype=object)

0

,Age,Import/Export,Measure Type,OBS,Ownership,Trade,Turnover,Year,Employees


We need to specify the units of the observations.

In [9]:
tidy['Unit'] = tidy['Measure Type'].map(lambda x: 'Businesses' if x == 'Count' else 'Percent')

And rename some columns.

In [10]:
tidy.rename(columns={'Employees': 'Employment',
                     'Ownership': 'Country of Ownership',
                     'OBS': 'Value',
                     'Trade': 'ONS ABS Trade'
                    }, inplace=True)

In [11]:
import urllib.request as request
import csv
import io
import requests

r = request.urlopen('https://raw.githubusercontent.com/ONS-OpenData/ref_trade/master/codelists/age-of-business.csv').read().decode('utf8').split("\n")
reader = csv.reader(r)
url="https://raw.githubusercontent.com/ONS-OpenData/ref_trade/master/codelists/age-of-business.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))
tidy = pd.merge(tidy, c, how = 'left', left_on = 'Age', right_on = 'Label')
tidy.columns = ['Age of Business' if x=='Notation' else x for x in tidy.columns]
r = request.urlopen('https://raw.githubusercontent.com/ONS-OpenData/ref_trade/master/codelists/exporter-and-importer-activity.csv').read().decode('utf8').split("\n")
reader = csv.reader(r)
url="https://raw.githubusercontent.com/ONS-OpenData/ref_trade/master/codelists/exporter-and-importer-activity.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))
tidy = pd.merge(tidy, c, how = 'left', left_on = 'Import/Export', right_on = 'Label')
tidy.columns = ['Export and Import Activity' if x=='Notation' else x for x in tidy.columns]


In [12]:
tidy = tidy[['Age of Business', 'Export and Import Activity','Measure Type','Value',
             'Country of Ownership','ONS ABS Trade','Turnover','Year','Employment','Unit']]

In [13]:
destinationFolder = Path('out')
destinationFolder.mkdir(exist_ok=True, parents=True)

tidy.to_csv(destinationFolder / ('observations.csv'), index = False)

In [14]:
scraper.dataset.family = 'trade'
scraper.dataset.comment = scraper.dataset.comment.replace('Importers and exporters of goods and services',
                                                          'Importers and exporters of trade goods and services')
from gssutils.metadata import THEME
scraper.dataset.theme = THEME['business-industry-trade-energy']
with open(destinationFolder / 'dataset.trig', 'wb') as metadata:
    metadata.write(scraper.generate_trig())